In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib widget
from collections import defaultdict
import geopandas as gpd
import textwrap
import os
import glob

pulje_størrelse = 7
min_pulje_størrelse = 5
max_teams_per_club = 2

## Data

### Udregn køretid mellem klubber 

In [72]:
def computeGEOtable(geo_data, server="127.0.0.1:5000", profile="car", annotations="duration,distance"): 
    # Create the coordinates list for sources and destinations
    address_coords = geo_data[['Hjemmebane lng', 'Hjemmebane lat']].apply(lambda row: f"{row['Hjemmebane lng']},{row['Hjemmebane lat']}", axis=1).tolist()
    coordinates_str = ";".join(address_coords)
    num_clubs = len(address_coords)
    sources_str = ";".join(map(str, range(num_clubs)))
    destinations_str = sources_str

    local_url = f"http://{server}/table/v1/{profile}/{coordinates_str}?sources={sources_str}&destinations={destinations_str}&annotations={annotations}&exclude=ferry"

    try:
        # Make the API request
        response = requests.get(local_url)
        response.raise_for_status()
        table_data = response.json()

        results = pd.DataFrame(columns=["Hjemmebane", "Udebane", "Køretid", "Afstand"])
        
        for i in tqdm(range(num_clubs)):
            for j in range(i, num_clubs): 
                if table_data['durations'][i][j] is not None:
                    duration = table_data['durations'][i][j] / 60 / 60  # hours
                    distance = table_data['distances'][i][j] / 1000      # km

                    source = geo_data["Hjemmebane"][i]
                    destination = geo_data["Hjemmebane"][j]
                    results.loc[len(results)] = [source, destination, duration, distance]
                    results.loc[len(results)] = [destination, source, duration, distance]
                    
    except requests.exceptions.RequestException as e:
        print(f"Failed to connect to OSRM server: {e}")

    return results

In [73]:
def forbered_data(path):
    # Indlæs data med hold_df
    hold_df = pd.read_excel(path)
    
    print("Klubber fjernet i datacleaning:")
    print(hold_df[hold_df["Hjemmebane lng"]<=1]["Hjemmebane"].unique())
    hold_df = hold_df[hold_df["Hjemmebane lng"]>1]
    hold_df = hold_df[hold_df["Hjemmebane lat"]>1]

    klubber_df = hold_df.drop_duplicates(subset="Hjemmebane").reset_index(drop=True)

    klubber_df = klubber_df[~klubber_df["Hjemmebane lat"].isna()]
    klubber_df = klubber_df.reset_index()
        
    # køretider_df = computeGEOtable(klubber_df)
    køretider_df = pd.read_csv("data/proc/køretider.csv")
    køretider_df.to_csv("data/proc/køretider.csv", index=False)
    
    return hold_df, klubber_df, køretider_df

### Beregning af køretider

In [74]:
def beregn_alle_køretider_i_række(række_df):
    """Beregner alle køretider i række"""
    alle_køretider_i_række = række_df[["Team id","Hjemmebane"]].merge(række_df[["Team id","Hjemmebane"]], how='cross')
    alle_køretider_i_række = alle_køretider_i_række.merge(køretider, left_on=["Hjemmebane_x", "Hjemmebane_y"], right_on=["Hjemmebane", "Udebane"], how='left', suffixes=('_left', '_right'))
    alle_køretider_i_række = alle_køretider_i_række.drop(columns=["Hjemmebane_x", "Hjemmebane_y"])
    alle_køretider_i_række = alle_køretider_i_række.rename(columns={"Team id_x": "Team id hjemmebane", "Team id_y": "Team id udebane"})
    alle_køretider_i_række = alle_køretider_i_række.drop_duplicates(subset=["Team id hjemmebane", "Team id udebane"])
    
    distance = defaultdict(int)

    for _, row in alle_køretider_i_række.iterrows():
        i = row["Team id hjemmebane"]
        j = row["Team id udebane"]
        distance[(i, j)] += row["Køretid"]
        distance[(j, i)] += row["Køretid"]
    
    return alle_køretider_i_række, distance

### Greedy heuristik

In [75]:
def saml_små_puljer(puljer):
    """ En række restpuljer kan opstå for de sidste puljer i den grådige algoritme
    Disse småpuljer samles i første omgang til puljer af præcis 7 hold og herefter en evt. rest.
    En evt rest fordeles i funktionen fordel_små_puljer()"""
    gyldige_puljer = []
    restpulje = []

    # Separate valid puljer and collect leftovers
    for pulje in puljer:
        if len(pulje) < pulje_størrelse:
            restpulje.extend(pulje)
        else:
            gyldige_puljer.append(pulje)

    # Split restpulje into chunks of size pulje_størrelse
    for i in range(0, len(restpulje), pulje_størrelse):
        gyldige_puljer.append(restpulje[i:i + pulje_størrelse])

    return gyldige_puljer

def håndhæv_max_to_klubber(puljer):
    """ Tjekker om nogen puljer indeholder mere end 2 hold fra samme klub
    I så fald foretages et tilfældigt bytte med et hold fra en anden pulje
    Byttet foretages under hensyn til ikke hermed at overstige max kravet"""
    for pulje in puljer:
        klubber_i_pulje = np.array([hold_til_klub[h] for h in pulje])
        vals, counts = np.unique(klubber_i_pulje, return_counts=True)
        overrepræsenterede_klubber = vals[counts > max_teams_per_club]
        overrepræsenterede_hold = [h for h in pulje if hold_til_klub[h] in overrepræsenterede_klubber]
        if overrepræsenterede_hold:
            for hold_ind in overrepræsenterede_hold[:]:

                for _, pulje_at_swappe in enumerate(puljer):
                    if pulje_at_swappe == pulje:
                        continue
                    klubber_i_swap = [hold_til_klub[h] for h in pulje_at_swappe]
                    if klubber_i_swap.count(hold_til_klub[hold_ind]) >= 2:
                        continue
                    hold_ud = pulje_at_swappe[0]
                    
                    pulje_at_swappe.remove(hold_ud)
                    pulje.remove(hold_ind)
                    
                    pulje.append(hold_ud)
                    pulje_at_swappe.append(hold_ind)

                    break 
    return puljer


def fordel_små_puljer(puljer):
    """Hvis antal_hold \\ antal_hold_pr_pulje ikke går op med heltalsdivision fordeles de tiloversblevne hold
    Hvis der er en rest på mindre end min_pulje_størrelse fordeles de resterende hold, således at puljer
    med pulje_størrelse + 1 dannes."""
    for pulje in puljer:
        if len(pulje) < min_pulje_størrelse:
            for hold_ind in pulje[:]:
                for _, pulje_at_swappe in enumerate(puljer):
                    if pulje_at_swappe == pulje:
                        continue
                    
                    pulje.remove(hold_ind)
                    pulje_at_swappe.append(hold_ind)

                    break 
            puljer.remove(pulje)
    return puljer
            

def grådig_fordeling(række_df):    
    """ Danner første bud på puljer gennem en grådig fordelingsalgoritme
    Gennemgår hold og vælger grådigt de 6 nærmeste hold, der ikke er fra samme klub
    
    For de sidst fordelte puljer vil der sjældent være 6 hold fra andre klubber tilbage.
    Her brydes puljerne op i puljer med mindre end 7 hold i hver pulje.
    Disse samles, hvorefter eventuelle overtrædelser af max_to_hold_fra_samme_klub-reglen håndteres ved tilfældige byt
    
    """ 
    hold_liste = række_df["Team id"].tolist()
    unused_teams = set(hold_liste)
    puljer_grådig = []

    while unused_teams:
        team = unused_teams.pop() 
        klub = hold_til_klub[team]

        possible_neighbors = alle_køretider_i_række[(alle_køretider_i_række["Team id hjemmebane"]==team) & \
                                (alle_køretider_i_række["Team id udebane"].isin(unused_teams)) & \
                                (alle_køretider_i_række["Udebane"] != klub)]

        possible_neighbors = possible_neighbors.drop_duplicates(subset="Udebane",keep="first")
        nearest_neighbors = possible_neighbors.sort_values(by="Køretid").head(pulje_størrelse - 1)
        nearest_neighbors = nearest_neighbors["Team id udebane"].tolist()
        pulje = [team] + nearest_neighbors
        puljer_grådig.append(pulje)
        
        for team in nearest_neighbors:
            unused_teams.remove(team)
            
    puljer_grådig = saml_små_puljer(puljer_grådig)
    puljer_grådig = håndhæv_max_to_klubber(puljer_grådig)
    puljer_grådig = fordel_små_puljer(puljer_grådig)
    puljer_grådig = håndhæv_max_to_klubber(puljer_grådig)
    
    
    return puljer_grådig


### Improvement algo helper functions

In [76]:
# ======================================================
# Distance utilities
# ======================================================

def make_distance_dict(alle_køretider_i_række):
    distance = defaultdict(float)
    for _, row in alle_køretider_i_række.iterrows():
        i = row["Team id hjemmebane"]
        j = row["Team id udebane"]
        distance[(i, j)] += row["Køretid"]
        distance[(j, i)] += row["Køretid"]
    return distance


# ======================================================
#  distance metrics
# ======================================================

def avg_team_distance(team, pulje, distance):
    return sum(distance[(team, t)] for t in pulje if t != team) / (len(pulje) - 1)

def max_team_distance(team, pulje, distance):
    return max(distance[(team, t)] for t in pulje if t != team)

def avg_pulje_distance(pulje, distance):
    return np.mean([avg_team_distance(t, pulje, distance) for t in pulje])

def max_avg_pulje_distance(pulje, distance):
    return max(avg_team_distance(t, pulje, distance) for t in pulje)

def avg_fordeling_distance(puljer, distance):
    return np.mean([avg_pulje_distance(p, distance) for p in puljer])

def max_fordeling_distance(puljer, distance):
    return max(max_avg_pulje_distance(p, distance) for p in puljer)


# ======================================================
#  outlier definitions
# ======================================================

def is_outlier(team, pulje, distance, outlier_factor):
    team_val = max_team_distance(team, pulje, distance)
    others = [max_team_distance(t, pulje, distance) for t in pulje if t != team]
    return team_val > outlier_factor * np.mean(others)


def pulje_is_outlier_idx(idx, puljer, distance, outlier_factor):
    this = max_avg_pulje_distance(puljer[idx], distance)
    others = [max_avg_pulje_distance(p, distance) for i, p in enumerate(puljer) if i != idx]
    return this > outlier_factor * np.mean(others)


# ======================================================
#  check constraints
# ======================================================

def club_count_ok(pulje, hold_til_klub, max_teams_per_club):
    counts = {}
    for t in pulje:
        c = hold_til_klub[t]
        counts[c] = counts.get(c, 0) + 1
        if counts[c] > max_teams_per_club:
            return False
    return True


def club_constraint_ok(old_a, old_b, new_a, new_b, i, j, a, b, puljer, distance, hold_til_klub,outlier_factor, max_teams_per_club):
    # Hard cap
    if not (club_count_ok(new_a, hold_til_klub, max_teams_per_club) and
            club_count_ok(new_b, hold_til_klub, max_teams_per_club)):
        return False

    # No duplicates → always allowed
    if len({hold_til_klub[t] for t in new_a}) == len(new_a) and \
       len({hold_til_klub[t] for t in new_b}) == len(new_b):
        return True

    # Individual outlier exception
    if is_outlier(i, old_a, distance, outlier_factor):
        return True
    if is_outlier(j, old_b, distance, outlier_factor):
        return True

    # Pulje-level outlier exception
    if pulje_is_outlier_idx(a, puljer, distance, outlier_factor):
        return True
    if pulje_is_outlier_idx(b, puljer, distance, outlier_factor):
        return True

    return False

def global_potential(puljer, distance, alpha):
    mean_term = avg_fordeling_distance(puljer, distance)
    max_term = max_fordeling_distance(puljer, distance)
    return (1 - alpha) * mean_term + alpha * (max_term / (mean_term + 1e-9))

### Improvement algo

In [77]:
def improvement_fordeling_soft_cap(puljer_init,distance,outlier_factor=1.5,fairness_factor=0.3):
    puljer = [p.copy() for p in puljer_init]
    current_phi = global_potential(puljer, distance, fairness_factor)

    improved = True
    while improved:
        improved = False

        for a in range(len(puljer)):
            for b in range(a + 1, len(puljer)):
                old_a, old_b = puljer[a], puljer[b]

                for i in old_a:
                    for j in old_b:
                        new_a = old_a.copy()
                        new_b = old_b.copy()

                        new_a.remove(i)
                        new_b.remove(j)
                        new_a.append(j)
                        new_b.append(i)

                        if not club_constraint_ok(
                            old_a, old_b, new_a, new_b, i, j, a, b,
                            puljer, distance, hold_til_klub, outlier_factor, max_teams_per_club):
                            continue

                        puljer_tmp = puljer.copy()
                        puljer_tmp[a] = new_a
                        puljer_tmp[b] = new_b

                        new_phi = global_potential(puljer_tmp, distance, fairness_factor)

                        if new_phi < current_phi - 1e-6:
                            puljer = puljer_tmp
                            current_phi = new_phi
                            improved = True
                            break
                    if improved:
                        break
                if improved:
                    break

    return puljer

def fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.5,fairness_factor=0.3):
    distance = make_distance_dict(alle_køretider_i_række)
    puljer_grådig = grådig_fordeling(række_df)
    return improvement_fordeling_soft_cap(puljer_grådig,distance,outlier_factor,fairness_factor)


### Visualisingsfunktioner

In [ ]:
def outline_points(polygons):
    xs, ys = [], []
    for poly in polygons:
        x, y = poly.boundary.xy
        xs.extend(x)
        ys.extend(y)
    return xs, ys

def add_denmark_background(fig, dk_x, dk_y, n_cols):
    for col in range(1, n_cols + 1):
        fig.add_trace(
            go.Scatter(
                x=dk_x,y=dk_y,mode="lines",
                line=dict(color="rgba(120,120,120,0.35)", width=1),
                hoverinfo="skip",showlegend=False),row=1, col=col)
        
def plot_puljer(puljer,region,division,hold_df):
    path_web = f"C:/Users/Lenovo/my-site/public/pulje_plots/Herre_{region}_{division}"
    
    ### Plot danmark
    world = gpd.read_file("dk/ne_110m_admin_0_countries.shp")
    
    denmark = world[world["NAME"] == "Denmark"].to_crs(epsg=4326)
    geom = denmark.geometry.iloc[0]
    polygons = [p for p in geom.geoms]

    dk_x, dk_y = outline_points(polygons)
    avg_distance = avg_fordeling_distance(puljer,distance)
    fig = make_subplots(rows=1,cols=2,subplot_titles=["Klubber", f"Puljefordeling"],horizontal_spacing=0.08)

    x_min, x_max = 7.5, 12.8
    y_min, y_max = 54.4, 57.9

    # Denmark background on both plots
    for col in [1, 2]:
        fig.add_trace(
            go.Scatter(x=dk_x,y=dk_y,mode="lines",line=dict(color="rgba(120,120,120,0.35)", width=1),
                hoverinfo="skip",showlegend=False),row=1,col=col)

    # Load klubber og team ids
    klubber = række_df["Team id"].map(hold_til_klub).values
    customdata = np.column_stack([række_df["Team id"].values])
    
    # plot all teams, no pulje clustering
    fig.add_trace(
        go.Scatter(x=række_df["Hjemmebane lng"],y=række_df["Hjemmebane lat"],
            mode="markers",showlegend=False,marker=dict(size=8,color="rgba(80,80,80,0.6)"),
            customdata=customdata,
                text=klubber,
                hovertemplate=
                    "<b>Hold id:</b> %{customdata}<br>"
                    "<b>Klub:</b> %{text}"
                    "<extra></extra>"
            ),row=1,col=1)

    # Find puljer med gengangere og små puljer
    puljer_med_gengangere = []
    gengangere = []
    små_puljer = []
    str_af_små_puljer = []
    for p_idx, pulje in enumerate(puljer, start=1):
        df = hold_df[hold_df["Team id"].isin(pulje)]
        
        klubber = df["Team id"].map(hold_til_klub).values
        vals, counts = np.unique(klubber, return_counts=True)
        klubber_med_to_hold = vals[counts>1]
        
        if len(klubber_med_to_hold)>0:
                gengangere.append(klubber_med_to_hold)
                puljer_med_gengangere.append(p_idx)
            
        if len(klubber) < pulje_størrelse:
            små_puljer.append(p_idx)
            str_af_små_puljer.append(len(pulje))
            
            
    # plot pulje clustering
    for p_idx, pulje in enumerate(puljer, start=1):
        df = hold_df[hold_df["Team id"].isin(pulje)]
        avg_cost = avg_pulje_distance(pulje,distance)
        max_dist = max_avg_pulje_distance(pulje,distance)

        color = f"rgba({(p_idx*25)%255},{(p_idx*60)%255},{(p_idx*110)%255},0.8)"
        lg = f"pulje_{p_idx}"

        klubber = df["Team id"].map(hold_til_klub).values
        customdata = np.column_stack([df["Team id"].values])
        
        fig.add_trace(
            go.Scatter(
                x=df["Hjemmebane lng"],
                y=df["Hjemmebane lat"],
                mode="markers",
                name=f"Pulje {p_idx}",
                legendgroup=lg,
                showlegend=True,
                marker=dict(size=10, opacity=0.6, color=color),
                customdata=customdata,
                text=klubber,
                meta=p_idx,
                hovertemplate=
                    "<b>Pulje:</b> %{meta}<br>"
                    "<b>Hold id:</b> %{customdata}<br>"
                    "<b>Klub:</b> %{text}"
                    "<extra></extra>"),row=1,col=2)
        
        # Pulje annotation (centroid)
        fig.add_trace(
            go.Scatter(x=[df["Hjemmebane lng"].mean()],y=[df["Hjemmebane lat"].mean()],mode="text",
                text=[f"{p_idx} ({avg_cost:.1f}, {max_dist:.1f})"],textfont=dict(color=color, size=12),
                hoverinfo="skip",showlegend=False,legendgroup=lg),row=1,col=2)

    # Layout & axis locking
    fig.update_layout(
         title={'text': f"Region {region}, {division}",'xanchor': 'center','x':0.5},
        title_font_weight = 600,
        hovermode="closest",height=650,width=800,
        legend=dict(itemclick="toggleothers",itemdoubleclick="toggle"),
        paper_bgcolor = "linen")

    for i in [1, 2]:
        fig.update_layout(
            **{
                f"xaxis{i}": dict(range=[x_min, x_max], fixedrange=True),
                f"yaxis{i}": dict(range=[y_min, y_max], fixedrange=True),})
            
            
    ### annoter plots med info om små puljer og gengangere
    klubnavne = [g[0] for g in gengangere]
    parts = [f"Pulje {p} ({k})"for p, k in zip(puljer_med_gengangere, klubnavne)]
    
    def wrap_annotation_text(text, line_width=150):
            return "<br>".join(textwrap.wrap(text, line_width))
    
    if len(parts) > 0:
        note = ("<b>Puljer med 2 hold fra samme klub:</b> "+ ", ".join(parts))
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper",font = dict(size = 8),
                    x=0, y=-0.13, showarrow=False,align="left")
        path_web = path_web + "_gengangere"

    if len(små_puljer)>0:
        små_puljer = [str(s) for s in små_puljer]
        str_af_små_puljer = [str(s) for s in str_af_små_puljer]
        note = (f"<b> Puljer med færre end {pulje_størrelse} hold: </b> Pulje "+ ", ".join(små_puljer) + " med " + ", ".join(str_af_små_puljer) + " hold")
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper",font = dict(size = 8),
                    x=0, y=-0.07, showarrow=False,align="left")
        path_web = path_web + "_lille_pulje"
    

    fig.write_html(f"plots/{region}_{division}.html", include_plotlyjs="cdn", full_html=True)
    fig.write_html(path_web + ".html", include_plotlyjs="cdn", full_html=True)
    return fig

### Tilføj puljer som kolonne

In [79]:
def tilføj_pulje_kolonne(puljer,række_df):
    række_df.loc[:,"Pulje"] = "Ingen pulje"

    pulje_bogstaver = list(map(chr, range(97, 97 + len(puljer))))
    pulje_bogstaver = [b.capitalize() for b in pulje_bogstaver]

    for i in range(len(puljer)):
        pulje_bogstav = pulje_bogstaver[i]
        række_df.loc[række_df["Team id"].isin(puljer[i]),"Pulje"] = pulje_bogstav
        
    return række_df

## Kør program

### Dameliga

In [ ]:
# Sørg for at alle plots i mappen slettes og dannes på ny
files = glob.glob("C:/Users/Lenovo/my-site/public/pulje_plots/*")
[os.remove(f) for f in files]

# load klubber, hold og køretider
hold_df, klubber_df, køretider = forbered_data("C:/Users/Lenovo/Desktop/PadelPuljer/data/raw/Hold med hjemmebane og koordinater.xlsx")

# vælg række
ligaer = hold_df["Liga"].unique()
regioner = hold_df["Region"].unique()
divisioner = hold_df["Division"].unique()

rækker = pd.DataFrame()


for liga in ligaer:
    print(f"Beregner puljefordelinger for {liga}...")
    liga = hold_df[hold_df["Liga"]!="DPF Ligaen"]
        
    for region in regioner:
        for division in divisioner:
            
            række_df = liga[(liga["Region"]==region) & (liga["Division"]==division)].copy()

            if len(række_df) == 0:
                continue
            
            print(f"Beregner puljefordeling for region {region}, division {division}...")

            alle_køretider_i_række, distance = beregn_alle_køretider_i_række(række_df)
            hold_til_klub = række_df.set_index("Team id")["Hjemmebane"].to_dict()

            # Fordel puljer
            puljefordeling = fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.5,fairness_factor=0.4)
            række_df = tilføj_pulje_kolonne(puljefordeling, række_df)
            plot_puljer(puljefordeling,region, division, hold_df)
            rækker = pd.concat([rækker,række_df])
            
rækker.to_excel(f"C:/Users/Lenovo/my-site/public/rækker.xlsx")


Klubber fjernet i datacleaning:
['GreenTEC Padel' 'Mors Padel' 'Skagen/Frederikshavn Padel'
 'Odense City Padel / Padelklub Odense City'
 'Dansk Gymnastikforening Flensborg' 'ViPadel Aarhus' 'ProPadel Denmark'
 'Uni10' 'Vallensbæk Tennis og Padel klub' 'ViPadel - Slagelse'
 'HYPE Padel Club']
Beregner puljefordeling for region Vest, division Serie 1 - A...
Beregner puljefordeling for region Vest, division Serie 4 - A...
Beregner puljefordeling for region Vest, division Serie 2 - A...
Beregner puljefordeling for region Vest, division Serie 5 - A...
Beregner puljefordeling for region Vest, division 2. Division - A...
Beregner puljefordeling for region Vest, division Danmarksserien - A...
Beregner puljefordeling for region Vest, division Serie 3 - A...


### Herreligaen

In [ ]:
# Sørg for at alle plots i mappen slettes og dannes på ny
files = glob.glob("C:/Users/Lenovo/my-site/public/pulje_plots/*")
[os.remove(f) for f in files]

# load klubber, hold og køretider
hold_df, klubber_df, køretider = forbered_data("C:/Users/Lenovo/Desktop/PadelPuljer/data/raw/Hold med hjemmebane og koordinater.xlsx")
# vælg række
herrehold = hold_df[hold_df["Liga"]=="DPF Ligaen"]
regioner = herrehold["Region"].unique()
divisioner = herrehold["Division"].unique()

rækker = pd.DataFrame()

for region in regioner:
    for division in divisioner:
        række_df = herrehold[(herrehold["Region"]==region) & (herrehold["Division"]==division)].copy()

        if len(række_df) == 0:
            continue
        
        print(f"Beregner puljefordeling for region {region}, division {division}...")

        alle_køretider_i_række, distance = beregn_alle_køretider_i_række(række_df)
        hold_til_klub = række_df.set_index("Team id")["Hjemmebane"].to_dict()

        # Fordel puljer
        puljefordeling = fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.5,fairness_factor=0.4)
        række_df = tilføj_pulje_kolonne(puljefordeling, række_df)
        plot_puljer(puljefordeling,region, division, hold_df)
        rækker = pd.concat([rækker,række_df])
        
rækker.to_excel(f"C:/Users/Lenovo/my-site/public/rækker.xlsx")


In [ ]:
def max_hold_samme_klub(pulje):
    """ Returnerer antal gange den mest repræsenterede klub er repræsenteret i en pulje"""
    klubber = [hold_til_klub[h] for h in pulje]
    _, counts = np.unique(klubber, return_counts=True)
    return counts.max()

def metrics_for_fordeling(puljer):
    """ Returnerer en række metrikker for en puljefordeling"""
    avg_pulje = avg_fordeling_distance(puljer,distance)
    max_hold = max_fordeling_distance(puljer,distance)
    avg_pr_pulje = [avg_pulje_distance(p,distance) for p in puljer]
    max_same_club = [max_hold_samme_klub(p) for p in puljer]
    
    return {
        "Antal puljer": len(puljer),
        "Gns. køretid (pulje-gennemsnit)": avg_pulje,
        "Min gns. køretid (bedste pulje)": min(avg_pr_pulje),
        "Max gns. køretid (værste pulje)": max(avg_pr_pulje),
        "Maks. køretid (enkelt hold)": max_hold,
        "Puljer med 2 hold fra samme klub": sum(m == 2 for m in max_same_club),
        "Puljer med 3 hold fra samme klub": sum(m == 3 for m in max_same_club),
    }

df_metrics = pd.DataFrame({
    "Outlier factor 1.6": metrics_for_fordeling(puljefordeling)
})
df_metrics

,Outlier factor 1.6
Antal puljer,9.000000
Gns. køretid (pulje-gennemsnit),1.828695
Min gns. køretid (bedste pulje),1.017889
Max gns. køretid (værste pulje),2.952800
Maks. køretid (enkelt hold),3.718991
Puljer med 2 hold fra samme klub,4.000000
Puljer med 3 hold fra samme klub,0.000000
